In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:

# 创建坐标框架
axis_pcd = o3d.geometry.TriangleMesh.create_coordinate_frame(size=20.0, origin=np.array([0, 0, 0]))


In [5]:
def extract_points_and_compute_centroid(vertices, target_z, tolerance):
    """
    提取 z 坐标接近 target_z 的点集，计算质心，并可视化点集和质心。
    
    参数:
        vertices: 点集数组，形状为 (N, 3)，每一行是一个点的 (x, y, z) 坐标。
        target_z: 目标 z 值。
        tolerance: 容忍度范围，筛选 z 坐标在 [target_z - tolerance, target_z + tolerance] 内的点。
    
    返回:
        centroid: 质心坐标，形状为 (3,) 的数组，表示 (x, y, z) 坐标。
    """
    # 提取 z 坐标接近 target_z 的点
    z_near_target = vertices[
        (vertices[:, 2] >= target_z - tolerance) & 
        (vertices[:, 2] <= target_z + tolerance)
    ]

    # 计算质心
    centroid = np.mean(z_near_target, axis=0)  # 对每一列求平均值，得到质心的 (x, y, z) 坐标
    
    # 可视化
    # 创建点云对象
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(z_near_target)  # 设置点云的点
    point_cloud.paint_uniform_color([0, 1, 0])  # 设置点云颜色为绿色
    
    # 创建质心点
    centroid_point = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)  # 创建一个球体表示质心
    centroid_point.translate(centroid)  # 将球体移动到质心位置
    centroid_point.paint_uniform_color([1, 0, 0])  # 设置质心颜色为红色
    
    
    # 可视化点云和质心
    o3d.visualization.draw_geometries([point_cloud, centroid_point])
    
    return centroid



In [ ]:
# import numpy as np
# import open3d as o3d

# def extract_points_and_compute_centroid(vertices, target_z, tolerance):
#     # 提取 z 坐标接近 target_z 的点
#     z_near_target = vertices[
#         (vertices[:, 2] >= target_z - tolerance) & 
#         (vertices[:, 2] <= target_z + tolerance)
#     ]
    
#     # 计算质心
#     centroid = np.mean(z_near_target, axis=0)  # 对每一列求平均值，得到质心的 (x, y, z) 坐标
    
#     # 可视化
#     # 创建点云对象
#     point_cloud = o3d.geometry.PointCloud()
#     point_cloud.points = o3d.utility.Vector3dVector(z_near_target)  # 设置点云的点
#     point_cloud.paint_uniform_color([0, 1, 0])  # 设置点云颜色为绿色
    
#     # 创建质心点
#     centroid_point = o3d.geometry.TriangleMesh.create_sphere(radius=0.1)  # 创建一个球体表示质心
#     centroid_point.translate(centroid)  # 将球体移动到质心位置
#     centroid_point.paint_uniform_color([1, 0, 0])  # 设置质心颜色为红色
    
#     # 创建闭合曲线
#     # 计算点的排序，假设点集在某个平面上，按照方位角排序
#     angles = np.arctan2(z_near_target[:, 1] - centroid[1], z_near_target[:, 0] - centroid[0])
#     sorted_indices = np.argsort(angles)
#     sorted_points = z_near_target[sorted_indices]
    
#     # 创建 LineSet 对象
#     lines = np.array([[i, (i+1)%len(sorted_points)] for i in range(len(sorted_points))])
#     line_set = o3d.geometry.LineSet(
#         points=o3d.utility.Vector3dVector(sorted_points),
#         lines=o3d.utility.Vector2iVector(lines)
#     )
#     line_set.paint_uniform_color([0, 0, 1])  # 设置线条颜色为蓝色
    
#     # 可视化点云、质心和闭合曲线
#     o3d.visualization.draw_geometries([point_cloud, centroid_point, line_set])
    
#     return centroid

In [ ]:
target_z = 3.0
tolerance = 0.5
centroid1= extract_points_and_compute_centroid(vertices_teeth1, target_z, tolerance)
print("质心坐标:", centroid1)

In [ ]:
import numpy as np
import open3d as o3d
# 读取网格
file_path1 = r"D:\wzpdata\teeth4\source\teeth5.ply"
mesh_teeth1 = o3d.io.read_triangle_mesh(file_path1)
mesh_teeth1.compute_vertex_normals()                    
mesh_teeth1.paint_uniform_color([0.1, 0.7, 0.3])
# 提取mesh的点集
vertices_teeth1 = np.asarray(mesh_teeth1.vertices)


# 计算质心
centroid = np.mean(vertices_teeth1, axis=0)
# 反转每个顶点相对于质心的位置


#z_coords = vertices_teeth2[:, 2]  # 提取所有点的 z 坐标
# z_max = np.max(z_coords)  # 获取 z 坐标的最大值
# z_min = np.min(z_coords)  # 获取 z 坐标的最小值


# 定义步长和容忍度
step_size = 0.5  # 每次遍历的 z 坐标步长
tolerance = 0.3  # 容忍度范围

# 初始化一个列表来存储每一层的质心坐标
centroids = []
step_num =  6
# 在 z_min 和 z_max 之间循环遍历
z_min = centroid[2]- (step_num/2)*step_size
z_max = centroid[2]+ (step_num/2)*step_size

current_z = z_min
print(f"current_z: {current_z}")

while current_z <= z_max:  
    centroid = extract_points_and_compute_centroid(vertices_teeth1, current_z, tolerance)
    centroids.append(centroid)
    print(f"在 z = {current_z} 处的质心坐标: {centroid}")
    current_z += step_size

# 打印所有质心坐标
print("所有质心坐标:")
for i, centroid in enumerate(centroids):
    print(f"层次 {i + 1}: {centroid}")

current_z: -13.582796179939626
在 z = -13.582796179939626 处的质心坐标: [ -4.90256916 -31.0219131  -13.57846029]
在 z = -13.082796179939626 处的质心坐标: [ -4.40521518 -31.24307366 -13.0900352 ]
在 z = -12.582796179939626 处的质心坐标: [ -4.38184245 -31.31983347 -12.57355227]
在 z = -12.082796179939626 处的质心坐标: [ -4.13376424 -31.46202322 -12.07413716]
在 z = -11.582796179939626 处的质心坐标: [ -4.28096207 -31.69396867 -11.56962736]
在 z = -11.082796179939626 处的质心坐标: [ -4.39058524 -32.29052729 -11.10797489]
在 z = -10.582796179939626 处的质心坐标: [ -4.23027749 -33.00707692 -10.58182069]
所有质心坐标:
层次 1: [ -4.90256916 -31.0219131  -13.57846029]
层次 2: [ -4.40521518 -31.24307366 -13.0900352 ]
层次 3: [ -4.38184245 -31.31983347 -12.57355227]
层次 4: [ -4.13376424 -31.46202322 -12.07413716]
层次 5: [ -4.28096207 -31.69396867 -11.56962736]
层次 6: [ -4.39058524 -32.29052729 -11.10797489]
层次 7: [ -4.23027749 -33.00707692 -10.58182069]


In [100]:

# 创建点云对象表示质心
centroid_point_cloud = o3d.geometry.PointCloud()
centroid_point_cloud.points = o3d.utility.Vector3dVector(centroids)  # 设置点云的点
centroid_point_cloud.paint_uniform_color([1, 0, 0])  # 设置质心颜色为红色

# 可视化 mesh 和质心点
o3d.visualization.draw_geometries([mesh_teeth1, centroid_point_cloud])

In [101]:
import numpy as np
import open3d as o3d

# 拟合直线
# 使用最小二乘法拟合直线
centroid_mean = np.mean(centroids, axis=0)  # 质心点的均值点
centroids_centered = centroids - centroid_mean  # 中心化数据
U, S, VT = np.linalg.svd(centroids_centered)  # 奇异值分解
direction_vector = VT[0]  # 主方向向量（直线的方向）

# 直线的参数方程：P(t) = P0 + t * direction_vector
# P0 是质心点的均值点
P0 = centroid_mean

# 生成拟合直线上的点
t_values = np.linspace(-10, 10, 100)  # 参数 t 的范围
line_points = np.array([P0 + t * direction_vector for t in t_values])

# 创建点云对象表示原始质心点
centroid_point_cloud = o3d.geometry.PointCloud()
centroid_point_cloud.points = o3d.utility.Vector3dVector(centroids)
centroid_point_cloud.paint_uniform_color([1, 0, 0])  # 设置质心颜色为红色

# 创建线集对象表示拟合的直线
line_set = o3d.geometry.LineSet()
line_set.points = o3d.utility.Vector3dVector(line_points)
lines = np.array([[i, i + 1] for i in range(len(line_points) - 1)])  # 连接点成线
line_set.lines = o3d.utility.Vector2iVector(lines)
line_set.paint_uniform_color([0, 0, 1])  # 设置直线颜色为绿色

# 可视化原始质心点和拟合直线
o3d.visualization.draw_geometries([centroid_point_cloud, line_set,mesh_teeth1])

In [88]:
from sklearn.linear_model import LinearRegression

def fit_line(points, extension_factor=5):
    if points.size == 0:
        raise ValueError("No valid points provided for line fitting.")
    
    reg = LinearRegression()
    reg.fit(points[:, 1].reshape(-1, 1), points[:, [0, 2]])
    
    min_y, max_y = np.min(points[:, 1]), np.max(points[:, 1])
    extend_length = (max_y - min_y) * extension_factor
    extended_min_y = min_y - extend_length
    extended_max_y = max_y + extend_length
    
    y_vals = np.linspace(extended_min_y, extended_max_y, 100)
    xz_vals = reg.predict(y_vals.reshape(-1, 1))
    return np.column_stack((xz_vals[:, 0], y_vals, xz_vals[:, 1]))

In [89]:

fit_line_points = fit_line(np.array(centroids), extension_factor=2)

In [90]:
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(fit_line_points)  # 设置点

o3d.visualization.draw_geometries([mesh_teeth1, point_cloud])